In [1]:
from nipype import Node, Workflow, IdentityInterface, Function, MapNode,SelectFiles, DataSink
from nipype.interfaces import fsl
from nipype.algorithms.modelgen import SpecifyModel
import numpy as np
import os

In [2]:
def gather_subject_info(tsv_filename):
    
    import pandas as pd
    from nipype.interfaces.base import Bunch
    
    trialinfo = pd.read_table(tsv_filename)

    conditions = []
    onsets = []
    durations = []

    for group in trialinfo.groupby('Stimulus'):
        conditions.append(group[0])
        onsets.append(list(group[1].onset)) 
        durations.append(list(group[1].duration))

    subject_info = Bunch(conditions=conditions,
                          onsets=onsets,
                          durations=durations)

    return subject_info

In [3]:
data   = "/home/paradeisios/Desktop/nipype_output/preprocessing/"
output = "/home/paradeisios/Desktop/nipype_output/first_level/"

num_of_subjects = 1
num_of_runs = 2

subject_list = ["{:02d}".format(i) for i in range(1,num_of_subjects+1)]

TR = 2.5


cont1 = ['Congruent',   'T', ['congruent', 'incongruent'], [1, 0]]
cont2 = ['Incongruent', 'T', ['incongruent', 'congruent'], [0, 1]]
cont3 = ['Congruent > Incongruent', 'T', ['congruent', 'incongruent'], [1, -1]]
cont4 = ['Incongruent > Congruent', 'T', ['incongruent', 'congruent'], [1, -1,]]
contrasts = [cont1,cont2,cont3,cont4]

In [4]:
infosource = Node(IdentityInterface(fields=["subject_list"]), name="infosource")
infosource.iterables = [('subject_list', subject_list)]

In [5]:
templates = {"func":        "{subject_list}/smooth/{subject_list}/smooth_run_*/sub-{subject_list}_task-flanker_run-*_bold_maths_roi_mcf_warp_smooth.nii.gz",
             "real_params":  "{subject_list}/realign/{subject_list}/realign_*/sub-{subject_list}_task-flanker_run-*_bold_maths_roi_mcf.nii.gz.par",
             "conditions" : "{subject_list}/conditions/sub-{subject_list}_task-flanker_run-*_events.tsv"}

selectfiles = Node(SelectFiles(templates),name="selectfiles")
selectfiles.inputs.base_directory = data

In [6]:
substitutions = []
substitutions += [("_subject_list_{:02d}".format(i), "{:02d}".format(i)) for i in range(1,num_of_subjects+1)]
substitutions += [("_modelestimate{:d}".format(i), "modelestimate_run_{:d}".format(i+1)) for i in range(0,num_of_runs)]
substitutions += [("_modelgen{:d}".format(i), "modelgen_run_{:d}".format(i+1)) for i in range(0,num_of_runs)]
substitutions += [("_level1design{:d}".format(i), "level1design_run_{:d}".format(i+1)) for i in range(0,num_of_runs)]

datasink = Node(DataSink(base_directory=output),name="datasink")
datasink.inputs.substitutions = substitutions
datasink.output = output

In [7]:
getsubjectinfo = MapNode(Function(input_names=["tsv_filename"],output_names=['subject_info'],
                                  function=gather_subject_info),
                                  iterfield = ["tsv_filename"],
                                  name='getsubjectinfo')


specify_model = MapNode(SpecifyModel(),name="specify_model",
                        iterfield=["functional_runs","subject_info","realignment_parameters"])
specify_model.inputs.input_units = 'secs'
specify_model.inputs.time_repetition = TR
specify_model.inputs.high_pass_filter_cutoff = 128.              




level1design = MapNode(fsl.Level1Design(), name="level1design",iterfield = ["session_info"])# make the model
level1design.inputs.bases = {'dgamma':{'derivs': False}}
level1design.inputs.interscan_interval = TR
level1design.inputs.model_serial_correlations=True
level1design.inputs.contrasts=contrasts

modelgen = MapNode(fsl.FEATModel(),name='modelgen',iterfield = ["fsf_file","ev_files"]) # transform to fsl model

modelestimate = MapNode(fsl.FILMGLS(), name='modelestimate',iterfield=["in_file","design_file","tcon_file"]) #run the model
modelestimate.inputs.smooth_autocorr=True 
modelestimate.inputs.threshold = 10

In [8]:
first_level_analysis = Workflow(name="first_level_analysis")

first_level_analysis.connect(infosource, "subject_list", selectfiles, "subject_list")
first_level_analysis.connect(selectfiles, "conditions", getsubjectinfo, "tsv_filename")

first_level_analysis.connect(selectfiles, "func", specify_model, "functional_runs")
first_level_analysis.connect(selectfiles, "real_params", specify_model, "realignment_parameters")
first_level_analysis.connect(getsubjectinfo,"subject_info",specify_model,"subject_info")

first_level_analysis.connect(specify_model,"session_info",level1design,"session_info")

first_level_analysis.connect(level1design,"fsf_files",modelgen,"fsf_file")
first_level_analysis.connect(level1design,"ev_files",modelgen,"ev_files")

first_level_analysis.connect(selectfiles, "func", modelestimate, "in_file")
first_level_analysis.connect(modelgen,"design_file",modelestimate,"design_file")
first_level_analysis.connect(modelgen,"con_file",modelestimate,"tcon_file")

first_level_analysis.connect(level1design,"fsf_files",datasink,"level1design.@fsf_files")
first_level_analysis.connect(level1design,"ev_files",datasink,"level1design.@ev_files")

first_level_analysis.connect(modelgen,"design_file",datasink,"model.@design_file")
first_level_analysis.connect(modelgen,"design_cov",datasink,"model.@design_cov")
first_level_analysis.connect(modelgen,"design_image",datasink,"model.@design_matrix")

first_level_analysis.connect(modelestimate,"copes",datasink,"modelestimate.@copes")
first_level_analysis.connect(modelestimate,"varcopes",datasink,"modelestimate.@varcopes")
first_level_analysis.connect(modelestimate,"tstats",datasink,"modelestimate.@tstats")
first_level_analysis.connect(modelestimate,"zstats",datasink,"modelestimate.@zstats")
first_level_analysis.connect(modelestimate,"dof_file",datasink,"modelestimate.@dof_file")

In [9]:
first_level_analysis.write_graph(graph2use='flat', format='png', simple_form=True,
                              dotfilename= os.path.join(output,"first_level_analysis_workflow.dot"))

210805-22:18:04,512 nipype.workflow INFO:
	 Generated workflow graph: /home/paradeisios/Desktop/nipype_output/first_level/first_level_analysis_workflow.png (graph2use=flat, simple_form=True).


'/home/paradeisios/Desktop/nipype_output/first_level/first_level_analysis_workflow.png'

In [10]:
first_level_analysis.run(plugin='MultiProc', plugin_args={'n_procs' : 1})

210805-22:18:04,580 nipype.workflow INFO:
	 Workflow first_level_analysis settings: ['check', 'execution', 'logging', 'monitoring']
210805-22:18:04,616 nipype.workflow INFO:
	 Running in parallel.
210805-22:18:04,620 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs ready. Free memory (GB): 6.74/6.74, Free processors: 1/1.
210805-22:18:04,693 nipype.workflow INFO:
	 [Node] Setting-up "first_level_analysis.selectfiles" in "/tmp/tmp808v4gie/first_level_analysis/_subject_list_01/selectfiles".
210805-22:18:04,702 nipype.workflow INFO:
	 [Node] Running "selectfiles" ("nipype.interfaces.io.SelectFiles")
210805-22:18:04,719 nipype.workflow INFO:
	 [Node] Finished "first_level_analysis.selectfiles".
210805-22:18:06,624 nipype.workflow INFO:
	 [Job 0] Completed (first_level_analysis.selectfiles).
210805-22:18:06,630 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs ready. Free memory (GB): 6.74/6.74, Free processors: 1/1.
210805-22:18:08,626 nipype.workflow INFO:
	 [

210805-22:18:28,770 nipype.workflow INFO:
	 [Node] Cached "_level1design1" - collecting precomputed outputs
210805-22:18:28,772 nipype.workflow INFO:
	 [Node] "_level1design1" found cached.
210805-22:18:28,788 nipype.workflow INFO:
	 [Node] Finished "first_level_analysis.level1design".
210805-22:18:30,647 nipype.workflow INFO:
	 [Job 3] Completed (first_level_analysis.level1design).
210805-22:18:30,651 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs ready. Free memory (GB): 6.74/6.74, Free processors: 1/1.
210805-22:18:32,650 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 2 jobs ready. Free memory (GB): 6.74/6.74, Free processors: 1/1.
210805-22:18:32,701 nipype.workflow INFO:
	 [Node] Setting-up "_modelgen0" in "/tmp/tmposr3g__c/first_level_analysis/_subject_list_01/modelgen/mapflow/_modelgen0".
210805-22:18:32,709 nipype.workflow INFO:
	 [Node] Running "_modelgen0" ("nipype.interfaces.fsl.model.FEATModel"), a CommandLine Interface with command:
feat_model ru

210805-22:33:47,657 nipype.interface INFO:
	 sub: /home/paradeisios/Desktop/nipype_output/first_level/level1design/_subject_list_01/_level1design0/ev_Realign1_0_3.txt -> /home/paradeisios/Desktop/nipype_output/first_level/level1design/01/level1design_run_1/ev_Realign1_0_3.txt
210805-22:33:47,661 nipype.interface INFO:
	 sub: /home/paradeisios/Desktop/nipype_output/first_level/level1design/_subject_list_01/_level1design0/ev_Realign2_0_4.txt -> /home/paradeisios/Desktop/nipype_output/first_level/level1design/01/level1design_run_1/ev_Realign2_0_4.txt
210805-22:33:47,665 nipype.interface INFO:
	 sub: /home/paradeisios/Desktop/nipype_output/first_level/level1design/_subject_list_01/_level1design0/ev_Realign3_0_5.txt -> /home/paradeisios/Desktop/nipype_output/first_level/level1design/01/level1design_run_1/ev_Realign3_0_5.txt
210805-22:33:47,669 nipype.interface INFO:
	 sub: /home/paradeisios/Desktop/nipype_output/first_level/level1design/_subject_list_01/_level1design0/ev_Realign4_0_6.txt ->

210805-22:33:47,786 nipype.interface INFO:
	 sub: /home/paradeisios/Desktop/nipype_output/first_level/modelestimate/_subject_list_01/_modelestimate0/varcope4.nii.gz -> /home/paradeisios/Desktop/nipype_output/first_level/modelestimate/01/modelestimate_run_1/varcope4.nii.gz
210805-22:33:47,794 nipype.interface INFO:
	 sub: /home/paradeisios/Desktop/nipype_output/first_level/modelestimate/_subject_list_01/_modelestimate1/varcope1.nii.gz -> /home/paradeisios/Desktop/nipype_output/first_level/modelestimate/01/modelestimate_run_2/varcope1.nii.gz
210805-22:33:47,797 nipype.interface INFO:
	 sub: /home/paradeisios/Desktop/nipype_output/first_level/modelestimate/_subject_list_01/_modelestimate1/varcope2.nii.gz -> /home/paradeisios/Desktop/nipype_output/first_level/modelestimate/01/modelestimate_run_2/varcope2.nii.gz
210805-22:33:47,805 nipype.interface INFO:
	 sub: /home/paradeisios/Desktop/nipype_output/first_level/modelestimate/_subject_list_01/_modelestimate1/varcope3.nii.gz -> /home/paradei